In [2]:
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset,load_metric

In [4]:
from transformers import pipeline
classifier_sent = pipeline('sentiment-analysis')
classifier_sent("how was that even possible")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9955288767814636}]

In [5]:
dataset = load_dataset('glue','sst2')
metric = load_metric('glue','sst2')

C:/Users/Brian/AppData/Local/Temp/xpython_10940/1482350923.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue','sst2')
D:\Software\anaconda3\envs\d2l\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [13]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [15]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
tokenizer("ECNU is a really wonderful place")
# tokenizer返回三个值

{'input_ids': [101, 14925, 11231, 2003, 1037, 2428, 6919, 2173, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
def preprocess_function(examples):
    return tokenizer(examples['sentence'], truncation = True, max_length=512)

In [21]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], [101, 3397, 2053, 15966, 1010, 2069, 4450, 2098, 18201, 2015, 102], [101, 2008, 7459, 2049, 3494, 1998, 10639, 2015, 2242, 2738, 3376, 2055, 2529, 3267, 102], [101, 3464, 12580, 8510, 2000, 3961, 1996, 2168, 2802, 102], [101, 2006, 1996, 5409, 7195, 1011, 1997, 1011, 1996, 1011, 11265, 17811, 18856, 17322, 2015, 1996, 16587, 2071, 2852, 24225, 2039, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [23]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [25]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [29]:
encoded_dataset['validation'][80]

{'sentence': "the draw ( for `` big bad love '' ) is a solid performance by arliss howard . ",
 'label': 1,
 'idx': 80,
 'input_ids': [101,
  1996,
  4009,
  1006,
  2005,
  1036,
  1036,
  2502,
  2919,
  2293,
  1005,
  1005,
  1007,
  2003,
  1037,
  5024,
  2836,
  2011,
  12098,
  6856,
  2015,
  4922,
  1012,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [31]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
import torch
device = torch.device("cuda")
print(torch.__version__)

2.1.0


In [35]:
import accelerate
import transformers

print("Current accelerate version:", accelerate.__version__)
print("Current transformers version:", transformers.__version__)

Current accelerate version: 0.30.1
Current transformers version: 4.41.1


In [37]:
from transformers import TrainingArguments
model = model.to(device)
batch_size = 16
args = TrainingArguments(
    "bert-base-uncased-finetuned-sst2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

In [39]:
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)  # 将概率最大的类别作为预测结果
    return metric.compute(predictions=predictions, references=labels)

In [41]:
from transformers import Trainer
trainer = Trainer(
    model,
    args,
    train_dataset = encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer = tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

{'loss': 0.3314, 'grad_norm': 10.634883880615234, 'learning_rate': 1.95249406175772e-05, 'epoch': 0.12}


{'loss': 0.2555, 'grad_norm': 22.65210723876953, 'learning_rate': 1.9049881235154395e-05, 'epoch': 0.24}


{'loss': 0.2276, 'grad_norm': 3.0995171070098877, 'learning_rate': 1.8574821852731593e-05, 'epoch': 0.36}


{'loss': 0.2086, 'grad_norm': 7.046559810638428, 'learning_rate': 1.809976247030879e-05, 'epoch': 0.48}


{'loss': 0.2006, 'grad_norm': 0.6357158422470093, 'learning_rate': 1.762470308788599e-05, 'epoch': 0.59}


...

{'eval_loss': 0.263675332069397, 'eval_accuracy': 0.9174311926605505, 'eval_runtime': 1.4554, 'eval_samples_per_second': 599.143, 'eval_steps_per_second': 37.79, 'epoch': 1.0}


100%|##################################################################################| 55/55 [00:01<00:00, 39.08it/s]


{'loss': 0.1461, 'grad_norm': 17.361488342285156, 'learning_rate': 1.5724465558194776e-05, 'epoch': 1.07}


{'loss': 0.123, 'grad_norm': 0.17007192969322205, 'learning_rate': 1.5249406175771972e-05, 'epoch': 1.19}


 24%|##################2                                                          | 5001/21050 [08:06<28:06,  9.52it/s]